# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#getting data: the dataset was uploaded to and is accesible through my github repo
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
#setting up workspace and experiment
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'expp03ht'

experiment=Experiment(ws, experiment_name)

In [4]:
# setting up IaaS where the experiment will run on it
ccname = "ccp03aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [6]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of 

        60   SparseNormalizer XGBoostClassifier             0:00:53       0.8790    0.8811
        61   SparseNormalizer LightGBM                      0:00:43       0.8790    0.8811
        62   StandardScalerWrapper LightGBM                 0:00:49       0.8790    0.8811
        63   SparseNormalizer XGBoostClassifier             0:00:40       0.8790    0.8811
        64   StandardScalerWrapper LightGBM                 0:00:32       0.8790    0.8811
        65                                                  0:00:22          nan    0.8811
        66                                                  0:00:12          nan    0.8811
        68    StackEnsemble                                 0:01:43       0.8816    0.8816
        67    VotingEnsemble                                0:02:05       0.8822    0.8822


{'runId': 'AutoML_88c58459-0355-48ec-875f-ac32d85d2aae',
 'target': 'ccp03aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-25T02:13:01.694559Z',
 'endTimeUtc': '2021-01-25T02:44:58.094994Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccp03aml',
  'AMLSettingsJsonString': '{"path":null,"name":"expp03ht","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-135954","workspace_name":"quick-starts-ws-135954","region":"southcentralus","compute_target":"ccp03aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
#import azureml.train.automl
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)
    

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: expp03ht,
Id: AutoML_88c58459-0355-48ec-875f-ac32d85d2aae_67,
Type: azureml.scriptrun,
Status: Completed)
f1_score_micro 0.8822470606608214
f1_score_macro 0.5062755954595995
precision_score_macro 0.8540100674822957
recall_score_weighted 0.8822470606608214
precision_score_micro 0.8822470606608214
average_precision_score_micro 0.8853349762034022
AUC_weighted 0.6159108810422379
matthews_correlation 0.15861315221590405
precision_score_weighted 0.8759923792783283
recall_score_micro 0.8822470606608214
recall_score_macro 0.5192093726150077
accuracy 0.8822470606608214
log_loss 0.3597178078221909
balanced_accuracy 0.5192093726150077
norm_macro_recall 0.038418745230015316
AUC_micro 0.903453935311194
AUC_macro 0.615910881042238
average_precision_score_macro 0.5677390130494524
average_precision_score_weighted 0.8281086062087157
f1_score_weighted 0.8326956201323578
weighted_accuracy 0.9804111334541978
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_88c58459-0355-48ec-875f-

In [10]:
import joblib
joblib.dump(value=fmodel, filename='model.pkl')

['model.pkl']

In [11]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='outputs/model.pkl', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-135954', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-135954'), name=p3automl_ht, id=p3automl_ht:1, version=1, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.8822470606608214'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [13]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='enviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://d7442aee-cb96-4dae-8d4e-6dee4432a08e.southcentralus.azurecontainer.io/score
http://d7442aee-cb96-4dae-8d4e-6dee4432a08e.southcentralus.azurecontainer.io/swagger.json


In [14]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

{"data": [{"id": 2921, "regfalle": 23774, "edad": 35, "deptoocuhe": 6, "tipoarma": 2, "pracaut": 1}, {"id": 1112, "regfalle": 19707, "edad": 20, "deptoocuhe": 14, "tipoarma": 9, "pracaut": 1}, {"id": 1064, "regfalle": 20238, "edad": 28, "deptoocuhe": 5, "tipoarma": 13, "pracaut": 1}, {"id": 2053, "regfalle": 23675, "edad": 24, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 2138, "regfalle": 23268, "edad": 20, "deptoocuhe": 1, "tipoarma": 1, "pracaut": 1}, {"id": 2136, "regfalle": 20835, "edad": 24, "deptoocuhe": 3, "tipoarma": 1, "pracaut": 1}, {"id": 790, "regfalle": 20229, "edad": 48, "deptoocuhe": 2, "tipoarma": 1, "pracaut": 0}, {"id": 2240, "regfalle": 21571, "edad": 56, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 1129, "regfalle": 19718, "edad": 20, "deptoocuhe": 14, "tipoarma": 9, "pracaut": 1}, {"id": 1819, "regfalle": 23550, "edad": 18, "deptoocuhe": 2, "tipoarma": 1, "pracaut": 1}]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [16]:
# Print results from the inference
print(response.text)

"{\"result\": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}"


In [17]:
# Print original labels
print(lblhomic2018)

2920    0
1111    0
1063    0
2052    0
2137    0
2135    1
789     0
2239    0
1128    0
1818    0
Name: sexo, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
print(webservice.get_logs())

2021-01-25T03:12:43,860479890+00:00 - gunicorn/run 
2021-01-25T03:12:43,863288914+00:00 - rsyslog/run 
2021-01-25T03:12:43,863370514+00:00 - nginx/run 
2021-01-25T03:12:43,864325022+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [19]:
#best practice: deleting webservice that it won't be used anymore
webservice.delete()

In [20]:
#best practice: deleting CCluster that it won't be used anymore
ccluster.delete()